In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

In [ ]:
acoes = ['CSMG3.SA', 'KLBN11.SA', 'BBSE3.SA', 'VALE3.SA', 'TAEE11.SA', 'BBAS3.SA']

ydata = yf.download(acoes, '2018-01-01', '2023-12-31', '1d');
dados = ydata["Adj Close"]

nova_ordem = ['BBAS3.SA', 'BBSE3.SA', 'CSMG3.SA', 'TAEE11.SA', 'KLBN11.SA', 'VALE3.SA']
dados = dados[nova_ordem]

In [ ]:
dados = dados *100 / dados.iloc[0]
display(dados)

In [ ]:
dados_chg = dados.pct_change()
dados_chg = dados_chg.fillna(0)

In [ ]:
#Calculo da correlação dos dados
dados_chg.corr()

In [ ]:
#####################################################################################################
#                                     ANALISE DE PORTIFÓLIO                                         #
#####################################################################################################


#                                a) GRAFICO DA FRONTEIRA EFICIENTE                                  #
#                                b) CRIANDO O GRÁFICO DE DRAWDOWN                                   #
#          c) A RENTABILIDADE ANUIZADA DOS ATIVOS QUE COMPÕE O PORTIFÓLIO E O PORTIFOLIO_1          #
#          d) A VOLATILIDADE ANUIZADA DOS ATIVOS QUE COMPÕE O PORTIFÓLIO E O PORTIFOLIO_1           #

In [ ]:
dados

In [ ]:
port_pesos = [0, 0, 0.5, 0, 0.5, 0]
dados['PORTIFOLIO_1'] = dados.dot(port_pesos)

In [ ]:
# D)

vol_aa = (dados_chg.std()*np.sqrt(252))*100
print("VOLATILIDADE ANUIZADA:\n", vol_aa.round(2))

In [ ]:
# B)

ddown = pd.DataFrame()

for ativo in dados.columns:
  list = []
  for ind in range(dados.count()[0]):
    list.append((dados[ativo].iloc[ind]/dados[ativo].iloc[:ind+1].max()-1)*100)
  ddown[ativo]=list

ddown['Data']=dados.index.values
ddown.set_index(keys = 'Data', inplace = True)

#display(ddown)
print(ddown.min())

ddown[['CSMG3.SA', 'KLBN11.SA', 'PORTIFOLIO_1']].plot(figsize = (15,5));

In [ ]:
dados=dados.drop(['PORTIFOLIO_1'], axis=1)
dados_chg=dados_chg.drop(['PORTIFOLIO_1'], axis=1)

In [ ]:
# A)

def calc_ret_vol(ativos, ativos_chg, port_pesos):
  port = ativos.dot(port_pesos)
  port_chg =port.pct_change()
  port_chg = port_chg.fillna(0)
  ret = ((port.iloc[-1]/port.iloc[0])**(1/5))-1
  vol = port_chg.std()*np.sqrt(252)
  return ret, vol

#Portfolio com 2 ativos: CSMG3 e KLBN11
points = []
min_vol_ret = [100, 0] #[vol, ret]
port_pesos = [0, 0, 0, 0, 0, 0]
for w in range(0, 101, 5):
  ret, vol = calc_ret_vol(dados, dados_chg, [0, 0,  w/100, 0, (1-w/100), 0])
  print(f"Aloc:{round(w/100, 2):.2f} {round(1-(w/100),2):.2f} Ret:{round(ret, 3):.3f} Vol:{round(vol, 3):.3f}")
  points.append([ret, vol])
  if vol < min_vol_ret[0]:
    min_vol_ret[0] = vol
    min_vol_ret[1] = ret
    port_pesos[2] = w/100
    port_pesos[4] = 1-w/100

lp = np.array(points).T
plt.scatter(lp[[1][:]],lp[[0][:]]);
plt.ylabel("Retorno");
plt.xlabel("Volatilidade");

plt.scatter(vol_aa['CSMG3.SA'], ret_aa['CSMG3.SA'], color='red');
plt.text(vol_aa['CSMG3.SA'], ret_aa['CSMG3.SA'], 'CSMG3');

plt.scatter(vol_aa['KLBN11.SA'], ret_aa['KLBN11.SA'], color='red');
plt.text(vol_aa['KLBN11.SA'], ret_aa['KLBN11.SA'], 'KLBN11');

plt.scatter(min_vol_ret[0], min_vol_ret[1], color='green');
plt.text(min_vol_ret[0], min_vol_ret[1], 'Min. Vol.');